In [1]:
import pandas as pd
from lda2vec.nlppipe import Preprocessor

# Data directory
data_dir ="data/"
# Where to save preprocessed data
clean_data_dir = "data/clean_data"
# Name of input file. Should be inside of data_dir
input_file = "restaurant.txt"
# Should we load pretrained embeddings from file
load_embeds = True

# Read in data file
df = pd.read_csv(data_dir+input_file, sep="\n")

# Initialize a preprocessor
P = Preprocessor(df, "texts", max_features=30000, maxlen=10000, min_count=0)

# Run the preprocessing on your dataframe
P.preprocess()

# Load embeddings from file if we choose to do so
if load_embeds:
    # Load embedding matrix from file path - change path to where you saved them
    embedding_matrix = P.load_para("../glove.6B.300d.txt")
else:
    embedding_matrix = None

# Save data to data_dir
P.save_data(clean_data_dir, embedding_matrix=embedding_matrix)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\ProgramData\Anaconda3\lib\site-packages\_pytest\fixtures.py:844: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
C:\ProgramData\Anaconda3\lib\site-packages\_pytest\fixtures.py:846: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)
Using TensorFlow backend.



---------- Tokenizing Texts ----------


699it [00:00, 1336.29it/s]


Removing 0 low frequency tokens out of 1428 total tokens

---------- Getting Skipgrams ----------


699it [00:00, 8985.67it/s]
C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\nlppipe.py:155: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  all_embs = np.stack(embeddings_index.values())


In [2]:
from lda2vec import utils, model
import tensorflow as tf
tf.reset_default_graph()

# Path to preprocessed data
data_path  = "data/clean_data"
# Whether or not to load saved embeddings file
load_embeds = True

# Load data from files
(idx_to_word, word_to_idx, freqs, pivot_ids,
 target_ids, doc_ids, embed_matrix) = utils.load_preprocessed_data(data_path, load_embed_matrix=load_embeds)

# Number of unique documents
num_docs = doc_ids.max() + 1
# Number of unique words in vocabulary (int)
vocab_size = len(freqs)
# Embed layer dimension size
# If not loading embeds, change 128 to whatever size you want.
embed_size = embed_matrix.shape[1] if load_embeds else 128
# Number of topics to cluster into
num_topics = 30
# Amount of iterations over entire dataset
num_epochs = 200
# Batch size - Increase/decrease depending on memory usage
batch_size = 4096
# Epoch that we want to "switch on" LDA loss
switch_loss_epoch = 0
# Pretrained embeddings value
pretrained_embeddings = embed_matrix if load_embeds else None
# If True, save logdir, otherwise don't
save_graph = False


# Initialize the model
m = model(num_docs,
          vocab_size,
          num_topics,
          embedding_size=embed_size,
          pretrained_embeddings=pretrained_embeddings,
          freqs=freqs,
          batch_size = batch_size,
          save_graph_def=save_graph)

W0628 20:37:25.743866 11060 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\Lda2vec.py:40: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0628 20:37:25.746827 11060 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\Lda2vec.py:42: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0628 20:37:25.770139 11060 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\Lda2vec.py:69: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0628 20:37:25.796068 11060 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\word_embedding.py:18: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0628 20:37:25.810032 11060 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\embedding_mixture.py:25: The nam

In [3]:
# Train the model
m.train(pivot_ids,
        target_ids,
        doc_ids,
        len(pivot_ids),
        num_epochs,
        idx_to_word=idx_to_word,
        switch_loss_epoch=switch_loss_epoch)


EPOCH: 1


InvalidArgumentError: indices[1156] = 1428 is not in [0, 1428)
	 [[node nce_loss/negative_sampling/nce_loss/embedding_lookup_1 (defined at C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\word_embedding.py:46) ]]

Errors may have originated from an input operation.
Input Source operations connected to node nce_loss/negative_sampling/nce_loss/embedding_lookup_1:
 nce_biases/read (defined at C:\ProgramData\Anaconda3\lib\site-packages\lda2vec\word_embedding.py:19)

Original stack trace for 'nce_loss/negative_sampling/nce_loss/embedding_lookup_1':
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
   

In [ ]:
len(idx_to_word)